In [1]:
import os
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import json
from geopy.distance import great_circle
import plotly.express as px
import plotly.graph_objects as go

In [2]:
existing_meteo = []
existing_meteo_new = []

for i in os.listdir("datasets/meteo/"):
    if not i.endswith(".csv"):
        continue
    existing_meteo.append(int(i.split(".")[0]))
    
for i in os.listdir("datasets/meteo_new/"):
    if not i.endswith(".csv"):
        continue
    existing_meteo.append(int(i.split(".")[0]))
    existing_meteo_new.append(int(i.split(".")[0]))

In [3]:
with open("datasets/processed_data/meteo_coords.pkl", "rb") as f:
    meteo_coords = pickle.load(f)
    meteo_coords = meteo_coords[[i in existing_meteo for i in meteo_coords["meteo_id"]]]
    
with open("datasets/processed_data/station_coords.pkl", "rb") as f:
    station_coords = pickle.load(f)

In [6]:
nearest_stantions = {}
nearest_meteo = {}

TOP_K_stantions = 15
TOP_K_meteo = 4

for stantion in station_coords[["station_id", "lat", "lon"]].values:
    stantion_id, lat, lon = stantion
    
    distances = []
    ids = []
    
    for _station_id, _lat, _lon in station_coords[["station_id", "lat", "lon"]].values:
        if _lon < lon:
            distance = round(great_circle((lat, lon), (_lat, _lon)).km, 1)
            distances.append(distance)
            ids.append(_station_id)
        
    
    ids_and_distances = zip(ids, distances)
    ids_and_distances = sorted(ids_and_distances, key=lambda x: x[1])[:TOP_K_stantions]
    nearest_stantions[stantion_id] = ids_and_distances
    
    
for stantion in station_coords[["station_id", "lat", "lon"]].values:
    stantion_id, lat, lon = stantion
    
    
    distances = [great_circle((lat, lon), _crd).km for _crd in station_coords[["lat", "lon"]].values]
    ids = meteo_coords["meteo_id"].astype(np.int32).tolist()
    
    ids_and_distances = zip(ids, distances)
    ids_and_distances = sorted(ids_and_distances, key=lambda x: x[1])[:TOP_K_meteo+1]
    nearest_meteo[stantion_id] = ids_and_distances
    
meteo_coords = meteo_coords[[i in existing_meteo_new for i in meteo_coords["meteo_id"]]]
for stantion in station_coords[["station_id", "lat", "lon"]].values:
    stantion_id, lat, lon = stantion
    
    if any([i[0] in existing_meteo_new for i in nearest_meteo[stantion_id]]):
        continue
    
    distances = [great_circle((lat, lon), _crd).km for _crd in station_coords[["lat", "lon"]].values]
    ids = meteo_coords["meteo_id"].astype(np.int32).tolist()
    
    ids_and_distances = zip(ids, distances)
    ids_and_distances = sorted(ids_and_distances, key=lambda x: x[1])
    nearest_meteo[stantion_id].append(ids_and_distances[0])

In [12]:
with open("temp_data/nearest_stantions.pkl", "wb") as f:
    pickle.dump(nearest_stantions, f)
    
with open("temp_data/nearest_meteo.pkl", "wb") as f:
    pickle.dump(nearest_meteo, f)